<a href="https://colab.research.google.com/github/eklz/Prediction-de-likes-Instagram/blob/master/InstaScrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/InstaScrap/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from selenium import webdriver
import datetime as dt
from bs4 import BeautifulSoup as bs
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np

In [0]:
class User():

  def __init__(username):

    self.username=username
    self.URL='http://www.instagram.com/'+username+'/?hl=fr'
    self.data=loadData()
    self.nbPubli= self.data['edge_owner_to_timeline_media']['count']
    self.followed_by=self.data['edge_followed_by']['count']
    self.follow=self.data['edge_follow']['count']
    self.business=self.data['business_category_name']

  def loadData(self):

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    browser = webdriver.Chrome('chromedriver',options=chrome_options)
    browser.get(self.URL)
    source = browser.page_source
    browser.quit()

    data = bs(source, 'html.parser')
    body = data.find('body')
    script = body.find('script', text=lambda t: t.startswith('window._sharedData'))
    page_json = script.text.split(' = ', 1)[1].rstrip(';')
    data = json.loads(page_json)
    data = data['entry_data']['ProfilePage'][0]['graphql']['user']
    return data

  def loadPictures(self):
    i=0
    for publi in data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['edges']:
      publi=publi['node']

      if publi['__typename'] == 'GraphImage':
        i+=1

        link='https://www.instagram.com'+'/p/'+publi['shortcode']+'/'
        analyse=publi['accessibility_caption']
        comments=publi['edge_media_to_comment']['count']
        likes=publi['edge_liked_by']['count']
        location_id=publi['location']['id']
        location_name=publi['location']['name']
        timestamp=publi['taken_at_timestamp']
        time=dt.datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')


